In [1]:
import sys
import os
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import math
import numpy as np
import skimage.measure

In [2]:
#mnist = input_data.read_data_sets('./mnist_data')
model_name = "mnist5CL"

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def reshape_batch(x):
    modified_x = []
    for image in x:
        i = np.reshape(image, (28, 28))
        modified_x.append(np.reshape(skimage.measure.block_reduce(i, (2, 2), np.max), (196)))
    modified_x = np.array(modified_x)
    return modified_x

Extracting ./mnist_data/train-images-idx3-ubyte.gz
Extracting ./mnist_data/train-labels-idx1-ubyte.gz
Extracting ./mnist_data/t10k-images-idx3-ubyte.gz
Extracting ./mnist_data/t10k-labels-idx1-ubyte.gz


In [3]:
data_train = np.load('blue2_train.npz')
data_trainX = data_train['X']
data_trainY = data_train['y']
data_test = np.load('blue2_test.npz')
data_testX = data_test['X']
data_testY = data_test['y']

In [4]:
data_test['X'].shape

(10000, 2352)

In [5]:
data_test['y'].shape

(10000,)

In [6]:
data_train['y'].shape

(60000,)

In [7]:
#mnist = input_data.read_data_sets('./mnist_data')
file_name = "mnist5CL.pb"

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def reshape_batch(x):
    modified_x = []
    for image in x:
        i = np.reshape(image, (28, 28))
        modified_x.append(np.reshape(skimage.measure.block_reduce(i, (2, 2), np.max), (196)))
    modified_x = np.array(modified_x)
    return modified_x

In [8]:
F = 5
C = 5
INPUT = 784*3
OUTPUT = 10
HFC = 50

In [9]:
x_in = tf.placeholder(tf.float32, [None, INPUT], name='input_op')

# W_conv = weight_variable([F, F, 1, C])
# b_conv = bias_variable([C])

#W_fc = weight_variable([INPUT, HFC])
#b_fc = bias_variable([HFC])

#W_o = weight_variable([HFC, OUTPUT])
#b_o = bias_variable([OUTPUT])

W_fc = weight_variable([INPUT, HFC])
b_fc = bias_variable([HFC])

W_fc1 = weight_variable([HFC, HFC])
b_fc1 = bias_variable([HFC])

W_fc2 = weight_variable([HFC, HFC])
b_fc2 = bias_variable([HFC])

W_fc3 = weight_variable([HFC, HFC])
b_fc3 = bias_variable([HFC])

W_fc4 = weight_variable([HFC, HFC])
b_fc4 = bias_variable([HFC])

W_fc5 = weight_variable([HFC, HFC])
b_fc5 = bias_variable([HFC])



W_o = weight_variable([HFC, OUTPUT])
b_o = bias_variable([OUTPUT])





#x_image = tf.reshape(x_in, [-1, int(math.sqrt(INPUT)), int(math.sqrt(INPUT)), 1])

# h_conv = tf.reshape(tf.nn.relu(conv2d(x_image, W_conv) + b_conv), [-1, INPUT*C])


h_fc = tf.nn.relu(tf.add(tf.matmul(x_in, W_fc), b_fc))
h_fc1 = tf.nn.relu(tf.add(tf.matmul(h_fc, W_fc1), b_fc1))
h_fc2 = tf.nn.relu(tf.add(tf.matmul(h_fc1, W_fc2), b_fc2))
h_fc3 = tf.nn.relu(tf.add(tf.matmul(h_fc2, W_fc3), b_fc3))
h_fc4 = tf.nn.relu(tf.add(tf.matmul(h_fc3, W_fc4), b_fc4))
h_fc5 = tf.nn.relu(tf.add(tf.matmul(h_fc4, W_fc5), b_fc5))


y = tf.nn.relu(tf.add(tf.matmul(h_fc5, W_o), b_o, name="output_op"))

y_ = tf.placeholder(tf.int64, [None])
cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=y)
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [ ]:
import models

x_in, y, cross_entropy, y_, _ = models.model_fn()

train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [ ]:
grap

In [ ]:
y

In [11]:
epochs = 10
batch_size = 32

N = data_trainX.shape[0]
batch_amount = epochs*N//batch_size

with tf.Session() as sess:
    epoch_in = 1
    tf.global_variables_initializer().run()
    for i in range(batch_amount):
        if (i*batch_size) % N == 0:
            print(epoch_in)
            #print('loss> {}'.format(sess.run(cross_entropy, feed_dict={x_in: batch_xs, y_: batch_ys})))
            #graph = tf.get_default_graph()
            #print(sess.run(graph.get_tensor_by_name('fc1/weight:0')))
            epoch_in += 1
        #batch_xs, batch_ys = mnist.train.next_batch(50)
        batch_xs = data_trainX[range(i*batch_size % N, i*batch_size % N + batch_size)]
        batch_ys = data_trainY[range(i*batch_size % N, i*batch_size % N + batch_size)]
        # modified_batch_xs = reshape_batch(batch_xs)
        sess.run(train_step, feed_dict={x_in: batch_xs, y_: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y, 1), y_)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    # modified_test_x = reshape_batch(test_x)

    print(sess.run(accuracy, feed_dict={x_in: data_testX, y_: data_testY}))
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['output_op']) 
    
    saver = tf.train.Saver()
    save_path = saver.save(sess, 'models/'+model_name+'.ckpt')
    with tf.gfile.GFile('models/' + model_name+'.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

1
2
3
4
5
6
7
8
9
10
0.9384
INFO:tensorflow:Froze 14 variables.
Converted 14 variables to const ops.


In [ ]:
! mkdir models

In [ ]:
! ls